In [19]:
# parts of code was taken from https://github.com/BogdanJeN/Geo/blob/main/Lab_06/lab_06.ipynb

import pandas as pd
import numpy as np
import rasterio
import geopandas as gpd
from shapely.geometry import Point, mapping
import json

In [20]:
# Load soil data
clay_path = '../soil_data/clay.tif'
sand_path = '../soil_data/sand.tif'
density_path = '../soil_data/density.tif'

clay_raster = rasterio.open(clay_path)
sand_raster = rasterio.open(sand_path)
density_raster = rasterio.open(density_path)

In [21]:
# Load field centroids
fields_path = '../soil_data/field_polygons.geojson'
fields = gpd.read_file(fields_path)

# Load moisture data
moisture_data = rasterio.open('../jpeg_soil_moisture.jpg')

# Create a list to store the field information
field_list = []

In [22]:
for index, row in fields.iterrows():
    # Get the field name and centroid coordinates
    field_id = row['id']
    field_name = row['Name']
    centroid = row['geometry'].centroid
    lat, lon = centroid.y, centroid.x

    # Get the pixel coordinates of the centroid
    row, col = clay_raster.index(lon, lat)

    # Extract soil data for the centroid
    clay = clay_raster.read(1)[row, col]
    sand = sand_raster.read(1)[row, col]
    density = density_raster.read(1)[row, col]

    # Get the pixel coordinates of the centroid for the moisture data
    row, col = sand_raster.index(lon, lat)

    # Extract moisture data
    moisture = moisture_data.read(1)[row, col]

    # Create a dictionary with the field information
    field_dict = {
        'type': 'Feature',
        'properties': {
            'id': field_id,
            'name': field_name,
            'soil': {
                'clay': float(clay),
                'sand': float(sand),
                'density': int(density)
            },
            'soil_moisture': int(moisture),
            'coordinates': {
                'lat': float(lat),
                'lng': float(lon)
            }
        },
        'geometry': mapping(centroid)
    }

    # Add the field dictionary to the list
    field_list.append(field_dict)

# Convert the field list to a GeoJSON FeatureCollection
feature_collection = {
    'type': 'FeatureCollection',
    'features': field_list
}

# Write the GeoJSON to a file
with open('field_data.geojson', 'w') as f:
    json.dump(feature_collection, f, indent=2)
